In [3]:
import pandas as pd
import numpy as np
from scipy.integrate import quad
import scipy.stats as stats
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import linprog
%matplotlib inline

In [9]:
def expected_return(asset_file_name, opening):
    # Construct the path to the data folder
    filepath = f"../data/{asset_file_name}"  

    # Read the CSV file
    asset = pd.read_csv(filepath)

    # Clean the columns by removing dollar signs and commas, and convert to float
    asset = asset[['Close/Last', 'Open', 'High', 'Low']].replace({'\$': '', ',': ''}, regex=True).astype(float)

    y = asset['Open']
    x = asset['Close/Last']

    value = x - y
    mean_value = sum(value) / len(value)
    mad = sum(abs(i - mean_value) for i in value) / len(value)

    data = np.vstack([x, y])

    # Create a Gaussian KDE model for the data
    kde = gaussian_kde(data, bw_method = 'silverman')

    # Create a grid for plotting the KDE
    x_grid, y_grid = np.mgrid[80:550:50j, 80:550:50j]
    grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()])

    # Evaluate the KDE on the grid
    z_density = kde(grid_points)
    z_grid = z_density.reshape(x_grid.shape) 


    # Define the grid over which to evaluate the joint density
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()

    x_grid = np.linspace(x_min-5, x_max+5, 100)
    y_grid = np.linspace(y_min-5, y_max+5, 100)

    X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
    positions = np.vstack([X_grid.ravel(), Y_grid.ravel()])

    # Evaluate the joint KDE at the grid points
    joint_pdf_values = kde(positions) 

    # Reshape the result to match the grid shape for easier plotting or analysis
    joint_pdf_grid = joint_pdf_values.reshape(X_grid.shape)

    # Estimate the marginal distribution of Y by summing over all values of X
    marginal_y = np.sum(joint_pdf_grid, axis=0) * (x_grid[1] - x_grid[0])

    # Choose a specific value of Y for conditioning (e.g., Y = 150)
    y_val = opening

    # Round the grid to 2 decimal places
    y_grid_rounded = np.round(y_grid, 4)

    # Find the index of the grid point closest to the target value, y_val
    y_idx = np.abs(y_grid_rounded - y_val).argmin()

    # Compute the conditional PDF of X given Y = y_val
    conditional_pdf = joint_pdf_grid[:, y_idx] / marginal_y[y_idx]

    # Calculate the expected value of X given Y = y_val
    expected_x_given_y = np.sum(x_grid * conditional_pdf * (x_grid[1] - x_grid[0]))

    return expected_x_given_y, mad

<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
/var/folders/nk/xk4mj8ts0ps645l4yh1nyfmh0000gp/T/ipykernel_77782/4271810603.py:9: SyntaxWarning: invalid escape sequence '\$'
  asset = asset[['Close/Last', 'Open', 'High', 'Low']].replace({'\$': '', ',': ''}, regex=True).astype(float)


In [10]:
filenames = ['AAPL - STOCK.csv', 'AMZN - STOCK.csv', 'BABA - STOCK.csv', 'GOOGL - STOCK.csv', 'MSFT - STOCK.csv',
             'AGG - BOND.csv', 'BND - BOND.csv', 'BNDX - BOND.csv', 'TLT - BOND.csv', 'VCIT - BOND.csv',
             'BTC - CRYPTO.csv', 'ETH - CRYPTO.csv', 'SOL - CRYPTO.csv', 'XRP - CRYPTO.csv']

asset_titles = ['Apple','Amazon','Ali baba','Google','Microsoft','AGG','BND','BNDX','TLT','VCIT','BTC','ETH','SOL','XRP']

In [11]:
## Apple, Amazon, Baba, Google, Microsoft, AGG, BND, BNDX, TLT, VCIT, BTC, ETH, SOL, USDT, XRP
Day_1_opening = [234.12, 239.02, 99.39, 195.56, 446.69, 97.58, 72.43, 48.96, 88.43, 80.93, 102007, 3124.87, 231.05, 3.0931]
Day_2_opening = [238.67, 237.14, 97.30, 198.00, 418.77, 97.62, 72.46, 49.08, 88.41, 80.88, 105498, 3196.48, 241.11, 3.122]
Day_3_opening = [247.19, 236.50, 102.00, 202.00, 418.98, 97.62, 72.48, 49.19, 88.38, 81.01, 104005, 3216.99, 235.92, 3.093]
Day_4_opening = [229.99, 234.06, 96.51, 200.69, 411.60, 97.34, 72.23, 49.32, 88.58, 80.50, 92749.80, 2452.90, 189.62, 2.2482]
Day_5_opening = [227.25, 239.01, 100.59, 203.39, 412.69, 97.02, 72.04, 49.19, 87.55, 80.38, 100069, 2798.57, 212.934, 2.7187]

In [12]:
Day_1_closing = [239.36, 237.07, 96.72, 195.41, 442.33, 97.46, 72.34, 48.98, 88.01, 80.77, 105361, 3195.29, 240.87, 3.1242]
Day_2_closing = [237.59, 234.64, 102.74, 200.87, 414.99, 97.60, 72.44, 49.12, 88.34, 80.93, 104197, 3223.77, 236.217, 3.0791]
Day_3_closing = [236.00, 237.68, 98.84, 204.02, 415.06, 97.40, 72.34, 49.17, 87.76, 80.80, 102381, 3301.14, 232.10, 3.05]
Day_4_closing = [228.01, 237.42, 98.61, 201.23, 410.92, 97.17, 72.17, 49.22, 88.16, 80.48, 100151, 2801.38, 213.47, 2.6755]
Day_5_closing = [232.80, 242.06, 102.35, 206.38, 412.37, 97.35, 72.27, 49.29, 88.43, 80.65, 98213.40, 2730.95, 205.95, 2.5107]

In [13]:
# Number of assets (5 stocks + 5 bonds + 4 cryptos = 14 assets)
n_assets = 14

stocks_opening_prices = Day_5_opening[0:5]
bonds_opening_prices = Day_5_opening[5:10] 
cryptos_opening_prices = Day_5_opening[10:14] 

expected_values_stocks = np.empty(5)
expected_values_bonds = np.empty(5)
expected_values_cryptos = np.empty(4)

risks_stocks = np.empty(5)
risks_bonds = np.empty(5)
risks_cryptos = np.empty(4)

for i in range(5):
    expected_values_stocks[i], risks_stocks[i] = expected_return(filenames[i], stocks_opening_prices[i])
    expected_values_bonds[i], risks_bonds[i] = expected_return(filenames[i+5], bonds_opening_prices[i])

for i in range(4):
    expected_values_cryptos[i], risks_cryptos[i] = expected_return(filenames[i+10], cryptos_opening_prices[i])

# Combine the assets data (risk will be used in constraint)
opening_prices = np.concatenate([stocks_opening_prices, bonds_opening_prices, cryptos_opening_prices])
expected_values = np.concatenate([expected_values_stocks, expected_values_bonds, expected_values_cryptos])
print(expected_values)
print(Day_5_closing)
risks = np.concatenate([risks_stocks, risks_bonds, risks_cryptos])
risks = risks / opening_prices

min_risk = np.min(risks)
max_risk = np.max(risks)

risks = (risks - min_risk) / (max_risk - min_risk)

# Budget (example)
budget = 150000

# New Objective Function: maximize sum of (x_i / opening_price_i * expected_value_i)
# No risk factor is considered anymore in the objective function
c = - ((expected_values - opening_prices)/ opening_prices)  # Negate to convert to minimization problem

# Risk constraint: sum(x_i / budget * risk_i) <= max_total_risk
max_total_risk = 0.9 # Example maximum risk threshold (you can adjust this value)

# Coefficients for the risk constraint
A_risk = risks / budget  # This gives the weights for the risk terms in the portfolio
b_risk = np.array([max_total_risk])  # The maximum allowable portfolio risk
5 

# Coefficients for the inequality constraints
A_budget = np.ones((1, n_assets))  # Sum of x_i <= budget
b_budget = np.array([budget])

# Asset-specific constraints (x_i <= 0.2 * budget for each asset)
A_investment = np.eye(n_assets)  # Identity matrix for each x_i <= 0.2 * budget
b_investment = np.full(n_assets, 0.2 * budget)

# Combine all the inequality constraints
A_all = np.vstack([A_budget, A_investment, A_risk])
b_all = np.hstack([b_budget, b_investment, b_risk])

# Bounds for each decision variable (x_i >= 0 for each asset)
x_bounds = [(0, None)] * n_assets  # Non-negativity

# Solve the LP problem
result = linprog(c, A_ub=A_all, b_ub=b_all, bounds=x_bounds, method='highs')
optimal_investments = result.x


for i in range(len(result.x)):
    if(optimal_investments[i] != 0):
        print(asset_titles[i], optimal_investments[i], "\n")
    expected_end = expected_values * (optimal_investments / opening_prices)
    actual_end = Day_5_closing * (optimal_investments / opening_prices)

print("Money invested", np.sum(optimal_investments))

print("End return", np.sum(np.sum(actual_end)))

[2.27813098e+02 2.41479835e+02 1.01379794e+02 2.03539987e+02
 4.14186122e+02 9.70039346e+01 7.18916686e+01 4.92631566e+01
 8.60654913e+01 8.06866115e+01 1.00103510e+05 2.80880180e+03
 2.07281550e+02 2.59713555e+00]
[232.8, 242.06, 102.35, 206.38, 412.37, 97.35, 72.27, 49.29, 88.43, 80.65, 98213.4, 2730.95, 205.95, 2.5107]
Amazon 30000.0 

Ali baba 30000.0 

Microsoft 30000.0 

VCIT 30000.0 

ETH 30000.0 

Money invested 150000.0
End return 150260.37136414085


In [14]:
end_return = np.array([148604, 151659, 149223, 153497, 151459])